## The Cython language

* The Cython language is a superset of the Python language -- nearly any Python code is valid Cython code, with very few edge cases.

* Cython is a [_creole programming language_](https://en.wikipedia.org/wiki/Creole_language) -- a stable (programming) language developed from a mixture of different languages.  In this case, the component languages are Python, C, and C++, with a few cython-specific constructs.

* It is designed to make it easy to write compiled CPython extension modules, and to fluidly mix in C and C++ constructs.

## Brief tour of Cython types

* Regular python variables are dynamically typed, and work just fine in Cython.
* Dynamically typed variables are simply Cython `object` types.
* Cython introduces the `cdef` keyword to declare a C-level construct.
* Cython supports the rest of the C type system--almost all beginning with `cdef` or a variant.  We won't be covering all of these.

## Simple types

In [ ]:
%load_ext Cython

In [ ]:
%%cython

# untyped objects are simply Python declarations like we know and love...
o = 1
print(o)

# and can be reassigned to a different type...
o = 'a'

# This print() function happens at extension module import time.
print(o)

## Integral types

In [ ]:
%%cython

# The panoply of C integer types and their modifiers...
# We typically get away with using just ints and longs...
# https://en.wikipedia.org/wiki/C_data_types
cdef:
    int i = 0
    unsigned long j = 1
    signed short k = -3
    long long ll = 1LL
    bint flag = True

print(i, j, k, ll, flag)

### Cython enforces static typing (either compile time or runtime, depending)

In [ ]:
%%cython

cdef int i = 0

# ...

i = 'qwerty'

## Floating point types

In [ ]:
%%cython

# doubles are preferred for compatibility with Python `float` types
# https://docs.python.org/3/c-api/float.html

cdef:
    float a = 1.0
    double b = -1.0
    long double c = 1e100
print(a, b, c)

## Complex types

In [ ]:
%%cython

# `double complex` is preferred for compatibility with Python's `complex` type:
# https://docs.python.org/3/c-api/complex.html

cdef:
    float complex fc = 1+1j
    double complex dc = 1+1j
    long double complex ldc = 1+1j

print(fc, dc, ldc)
print(fc.real, dc.imag, ldc.conjugate())

## String types

* Note that string types in Python are complicated by Py2 / Py3 differences.
* This tutorial is entirely in Python 3, so we do not have to go into the minutiae here.
* See [the fine manual](http://cython.readthedocs.io/en/latest/src/tutorial/strings.html#python-string-types-in-cython-code) or [other documentation](http://shop.oreilly.com/product/0636920033431.do) for more details.

### Statically declaring Python string types in Cython

In [ ]:
%%cython

cdef str s = "asdf"
cdef bytes b = b"jkl;"
print(s, b)

##### Cython allows static declaration of Python types!

### C "string"s

In [ ]:
%%cython

bb = b"asdf"
cdef char *buf = bb
print(bb, buf)

#### Pop quiz:

* What can we infer from the (runtime) warning generated here?
    ```
    warning: [...] Obtaining 'char *' from externally modifiable global Python value
    ```
* When working with C-level strings, what must we keep in mind to maintain Python guarantees?

**Strong recommendation**: Work with `str` and `bytes` objects only; avoid `char *` buffers unless you're interfacing with external C code.

In [ ]:
%%cython

# Why is this very, very bad?
a = b'a'
b = b'b'
cdef char *c = a + b

# ...and how can we address it?

# Again, it's best to avoid using char *'s unless absolutely necessary -- keep your
# Strings Python-level strings, and all will be copacetic.

## Other statically-declarable Python types

In [ ]:
%%cython

import datetime
cimport cpython.datetime # We'll cover the `cimport` keyword later

import array
cimport cpython.array

cdef:
    list lst = [1]
    dict dd = {'a': 'b'}
    set ss = set([1])
    frozenset fs = frozenset([1])
    cpython.datetime.datetime dt = datetime.datetime.now()
    cpython.array.array aa = array.array('i', [1, 2, 3])
    
print(lst, dd, ss, fs, dt, aa)

### Nota Bene:

* Everything on the LHS of the declaration is a C-level, statically typed entity.
* Everything on the RHS of the declaration is a Python object of the corresponding type.
* Cython enforces the static type at compile time -- a `list` variable can only ever be assigned to a `list` (or subclass).

## Declaring and manipulating pointers

**NOTE** Only do this if you have to -- fortunately we rarely have to drop to using or manipulating pointers in Cython unless we're interfacing with a C library.

In [ ]:
%%cython

cdef:
    int *a = NULL
    int b = 42

# Point `a` at `b`
a = &b

# Dereference `a` -- Note the use of `a[0]` -- we don't use `*a` to dereference!
print(a[0], b)

# Modify b via a:
a[0] = 137

# Confirm that both refer to same value:
print(a[0], b)